In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

from deepexplain.tensorflow import DeepExplain
from keras import backend as K
from keras.models import Model
import tensorflow as tf

/home/brian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Read in data

In [2]:
df = pd.read_csv('preprocessed-fico-biased.csv', index_col='Unnamed: 0') 

features = [x for x in df.columns.tolist() if x!='target']

X = df[features].values
y = df['target'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X_train.shape

(6606, 37)

# Identify 50-50 baseline

## Basic Neural Network

In [4]:
BATCH_SIZE = 100
EPOCHS = 20

**CAREFUL**: each neural network session must be trained in a separate kernel.

In [5]:

with tf.Session():
    model = Sequential()
    model.add(Dense(X_train.shape[1] // 2,input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              verbose=1,
              validation_data=(X_test, y_test))
    model.save('biased-data-model.h5')
    
    baseline_candidates = []

    for x in X_train:
        predicted = model.predict(np.array([x]))
        if 0.499 < predicted < 0.501:
            print(x)
            print(predicted)
            baseline_candidates.append(x)


Train on 6606 samples, validate on 3255 samples
Epoch 1/20
6606/6606 [==============================] - 1s 100us/step - loss: 0.7322 - acc: 0.4873 - val_loss: 0.6934 - val_acc: 0.5346
Epoch 2/20
6606/6606 [==============================] - 0s 53us/step - loss: 0.6754 - acc: 0.6022 - val_loss: 0.6636 - val_acc: 0.6203
Epoch 3/20
6606/6606 [==============================] - 0s 51us/step - loss: 0.6507 - acc: 0.6606 - val_loss: 0.6445 - val_acc: 0.6642
Epoch 4/20
6606/6606 [==============================] - 0s 54us/step - loss: 0.6355 - acc: 0.6829 - val_loss: 0.6335 - val_acc: 0.6786
Epoch 5/20
6606/6606 [==============================] - 0s 53us/step - loss: 0.6251 - acc: 0.6954 - val_loss: 0.6257 - val_acc: 0.6854
Epoch 6/20
6606/6606 [==============================] - 0s 52us/step - loss: 0.6179 - acc: 0.6974 - val_loss: 0.6201 - val_acc: 0.6903
Epoch 7/20
6606/6606 [==============================] - 0s 55us/step - loss: 0.6123 - acc: 0.7018 - val_loss: 0.6155 - val_acc: 0.6906
Epoch 

In [6]:
# first element produces .5002 output
baseline = baseline_candidates[0]
baseline = baseline.reshape(-1,1)
#print(baseline)

In [7]:
baseline.shape

(37, 1)

# Generate DeepLIFT Local Attributions

In [1]:
BATCH_SIZE = 100
EPOCHS = 20

In [2]:
with DeepExplain(session=K.get_session()) as de:
    # train
    model = load_model('biased-data-model.h5')
    score = model.evaluate(X_test, y_test, verbose=0)
    
    # explain
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)
    
    xs = X_train
    ys = y_train
    
    baseline = baseline.squeeze()
    deeplift_attributions = de.explain('deeplift', target_tensor * ys, input_tensor, xs, baseline=baseline)


NameError: name 'DeepExplain' is not defined

In [ ]:
deeplift_attributions

# Generate Integrated Gradients Local Attributions

* Must restart Kernal to run (need a new context)

In [ ]:
with DeepExplain(session=K.get_session()) as de:
    # train
    model = Sequential()
    model.add(Dense(X_train.shape[1] // 2,input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test))

    score = model.evaluate(X_test, y_test, verbose=0)
    
    # explain
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)
    
    xs = X_train
    ys = y_train
    
    intgrad_attributions = de.explain('intgrad', target_tensor * ys, input_tensor, xs, baseline=baseline)


In [ ]:
intgrad_attributions

# Save Local Attributions

In [ ]:
deeplift_df = pd.DataFrame(deeplift_attributions, columns = features)

In [ ]:
intgrad_df = pd.DataFrame(intgrad_attributions, columns = features)

In [ ]:
deeplift_df.head()

In [ ]:
deeplift_df.to_csv("data/deeplift_attributions-50-50-basline.csv")
intgrad_df.to_csv("data/intgrad_attributions-50-50-baseline.csv")